In [2]:
using TSSOS
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials
using QuadGK
using JuMP
using Random
using NLopt
using HDF5

## Read data from integral solver

In [4]:
N, K, Geom, Gen, Exp, S, Kin, Vext, Hcore, Ten, Enucl = h5open("results_Be.hdf5", "r") do file
    N = read(file["N"]);
    K = read(file["K"]);
    Geom = read(file["Geom"]);
    Gen = read(file["Gen"]);
    
    exp_group = file["Exp"];
    Exp = transpose([read(exp_group[string(i)]) for i in 0:(length(keys(exp_group))-1)]);
    
    S = read(file["S"]);
    Kin = read(file["Kin"]);
    Vext = read(file["Vext"]);
    Hcore = read(file["Hcore"]);
    Ten = read(file["Ten"]);
    Enucl = read(file["Enucl"]);
    
    return N, K, Geom, Gen, Exp, S, Kin, Vext, Hcore, Ten, Enucl;

end;

In [5]:
Ten[2,2,2,:]

3-element Vector{Float64}:
 0.10410026042398934
 0.497328057900321
 0.3696606574812221

In [6]:
Hcore

3×3 Matrix{Float64}:
 -7.94674  -1.23532  -1.46014
 -1.23532  -1.93572  -1.6356
 -1.46014  -1.6356   -1.65923

## Build Polynomial

In [8]:
Nvars = K*Int(N/2)

@polyvar x[1:Nvars]

X = reshape(x, K, Int(N/2))

poly = Enucl

# One-electron part
for a in 1:(Int(N/2))
    for i in 1:K
        for j in 1:K
            poly += 2 * Hcore[i, j] * X[i, a] * X[j, a]
        end
    end
end

# Two-electron part
for a in 1:(Int(N/2))
    for b in 1:(Int(N/2))
        for i in 1:K
            for j in 1:K
                for k in 1:K
                    for l in 1:K
                        term = (2*Ten[i,j,k,l] - Ten[i,l,k,j]) * X[i,a] * X[j,a] * X[k,b] * X[l,b]
                        poly += term
                    end
                end
            end
        end
    end
end

In [9]:
poly

2.284812265809389x₁⁴ + 0.7234327064233126x₁³x₂ + 1.2155305509638104x₁³x₃ + 1.3233174570704105x₁²x₂² + 2.0383417952058673x₁²x₂x₃ + 1.0658543010735315x₁²x₃² + 4.569624531618778x₁²x₄² + 0.7234327064233126x₁²x₄x₅ + 1.2155305509638104x₁²x₄x₆ + 2.3743512927608545x₁²x₅² + 3.4126149739846685x₁²x₅x₆ + 1.660735584359744x₁²x₆² + 0.4164010416959575x₁x₂³ + 1.079926849974829x₁x₂²x₃ + 0.9871035344755422x₁x₂x₃² + 0.7234327064233125x₁x₂x₄² - 2.102067671380888x₁x₂x₄x₅ - 1.3742731787788012x₁x₂x₄x₆ + 0.4164010416959575x₁x₂x₅² + 0.43642143978940795x₁x₂x₅x₆ + 0.24831454158942928x₁x₂x₆² + 0.3393655559487937x₁x₃³ + 1.2155305509638101x₁x₃x₄² - 1.3742731787788012x₁x₃x₄x₅ - 1.189762566572425x₁x₃x₄x₆ + 0.6435054101854211x₁x₃x₅² + 0.738788992886113x₁x₃x₅x₆ + 0.3393655559487937x₁x₃x₆² + 0.497328057900321x₂⁴ + 1.4786426299248885x₂³x₃ + 1.8620118446928904x₂²x₃² + 2.3743512927608545x₂²x₄² + 0.4164010416959575x₂²x₄x₅ + 0.643505410185421x₂²x₄x₆ + 0.994656115800642x₂²x₅² + 1.4786426299248885x₂²x₅x₆ + 0.9510187910169509x₂

In [10]:
pop = [poly];
for a in 1:Int(N/2)
    for b in a:Int(N/2)
        S_val = 0
        for i in 1:K
            for j in 1:K

                S_val = S_val + S[i,j]*X[i,a]*X[j,b]
    
            end
        end
        if a==b
            S_val += -1
        end
        push!(pop,S_val)
    end
end

## Minimize the polynomial

In [31]:
opt,sol,data = tssos_first(pop, variables(pop), 2; numeq=3, QUIET = true, solution = true)
    
previous_sol = sol
previous_opt = opt
    
while ~isnothing(sol)
    previous_sol = sol
    previous_opt = opt
            
    opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
end

*********************************** TSSOS ***********************************
TSSOS is launching...
optimum = -14.569303987661165
Global optimality certified with relative optimality gap 0.000000%!
No higher TS step of the TSSOS hierarchy!


In [33]:
previous_sol

6-element Vector{Float64}:
  0.9760010488420826
 -0.10862306355915559
 -0.397996861284255
 -0.29903089789110815
 -0.22581991885581248
 -0.7052827462678364

In [14]:
output_file = "POP_result_Be.txt"
open(output_file, "w") do io
    println(io, "Final results from POP\n")
    println(io, "Energy $previous_opt\n")
    println(io, "MO coefficients:")
    println(io, "First orbital: $(previous_sol[1]), $(previous_sol[2]), $(previous_sol[3])")
    println(io, "Second orbital: $(previous_sol[4]), $(previous_sol[5]), $(previous_sol[6])")
end

In [15]:
poly(previous_sol)

-14.569303993074977